In [2]:
import os

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\HP\\Desktop\\chicken-disease-classification'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from src.chicken_disease.constants import *
from src.chicken_disease.utils.common import read_yaml, create_directories

In [9]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH): 

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
    

In [11]:
import os
import urllib.request as request
import zipfile as zipfile
import tensorflow as tf
import time

In [25]:
import os
import time
import tensorflow as tf

class PrepareCallBack:
    def __init__(self, config: "PrepareCallbacksConfig"):
        self.config = config

    @property
    def _create_tb_callback(self):
        """Create a TensorBoard callback with timestamped log directory."""
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callback(self):
        """Create a ModelCheckpoint callback to save the best model."""
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_callbacks(self):
        """Return a list of TensorBoard and ModelCheckpoint callbacks."""
        return [
            self._create_tb_callback,
            self._create_ckpt_callback
        ]


In [26]:
try:
    # Load configs
    config = ConfigManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    
    # Prepare callback objects
    prepare_callback = PrepareCallBack(config=prepare_callbacks_config)
    
    # Get the list of callbacks
    callback_list = prepare_callback.get_callbacks()

except Exception as e:
    raise e


[2025-11-21 22:28:26,691: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-21 22:28:26,695: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-21 22:28:26,697: INFO: common: created directory at : artifacts]
[2025-11-21 22:28:26,698: INFO: common: created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2025-11-21 22:28:26,700: INFO: common: created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]


In [36]:
import time
time_stamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"Tensorboard logs at : {time_stamp}"

'Tensorboard logs at : 2025-11-21-22-36-07'

In [37]:
pass